# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv").set_index("City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Dryden,49.7833,-92.7503,17.16,63,100,4.12,CA,2023-07-15
1,Puerto Ayora,-0.7393,-90.3518,29.98,92,41,3.13,EC,2023-07-15
2,Arish,31.1316,33.7984,27.24,75,0,2.11,EG,2023-07-15
3,Ushuaia,-54.8000,-68.3000,1.81,69,40,10.80,AR,2023-07-15
4,Pionerskiy,54.9508,20.2275,20.97,57,98,6.67,RU,2023-07-15


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [54]:
# Narrow down cities that fit criteria and drop any results with null values
# Set limits for ideal weather conditions (temp in Celsius, humidity and cloudiness as percentage)
ideal_max_temp = 26.5
ideal_max_humidity = 50
ideal_max_cloudiness = 10

# Filter data frame for ideal weather conditions
city_data_df_filtered = city_data_df[(city_data_df["Max Temp"] < ideal_max_temp) & 
                                     (city_data_df["Humidity"] < ideal_max_humidity) & 
                                     (city_data_df["Cloudiness"] < ideal_max_cloudiness)]

# Drop any rows with null values
city_data_df_filtered = city_data_df_filtered.dropna()

# Display sample data
city_data_df_filtered.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
73,Andalgala,-27.5819,-66.3165,17.73,25,0,2.24,AR,2023-07-15
101,Coronel Du Graty,-27.6804,-60.9146,20.57,41,0,1.19,AR,2023-07-15
125,Zaragoza,41.6561,-0.8773,24.25,49,0,13.38,ES,2023-07-15
129,Bilibino,68.0546,166.4372,23.55,47,8,1.19,RU,2023-07-15
175,Carnarvon,-24.8667,113.6333,13.82,39,0,7.01,AU,2023-07-15


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filtered.copy().reset_index()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Andalgala,AR,-27.5819,-66.3165,25,
1,Coronel Du Graty,AR,-27.6804,-60.9146,41,
2,Zaragoza,ES,41.6561,-0.8773,49,
3,Bilibino,RU,68.0546,166.4372,47,
4,Carnarvon,AU,-24.8667,113.6333,39,
5,Konya,TR,38.1667,32.5000,34,
6,Karratha,AU,-20.7377,116.8463,44,
7,South Hedland,AU,-20.4089,118.5983,34,
8,General Pico,AR,-35.6566,-63.7568,35,
9,Wuwei,CN,37.9281,102.6414,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Andalgala - nearest hotel: Residencial Ayucaba
Coronel Du Graty - nearest hotel: No hotel found
Zaragoza - nearest hotel: Hotel Pilar Plaza
Bilibino - nearest hotel: No hotel found
Carnarvon - nearest hotel: No hotel found
Konya - nearest hotel: No hotel found
Karratha - nearest hotel: Karratha International Hotel
South Hedland - nearest hotel: Port Haven
General Pico - nearest hotel: Euskadi Hotel
Wuwei - nearest hotel: Hepingli Boutique Hotel
Raton - nearest hotel: Travelodge
Calingasta - nearest hotel: Nora Hôtel
Laojunmiao - nearest hotel: No hotel found
San Juan - nearest hotel: Hotel San Francisco
El Burgo De Osma - nearest hotel: Hotel Termal Burgo de Osma
Tvardita - nearest hotel: No hotel found
Kantagi - nearest hotel: No hotel found
Chibougamau - nearest hotel: No hotel found
Baiyin - nearest hotel: 红鹭宾馆
Kasongo-Lunda - nearest hotel: No hotel found
Zapala - nearest hotel: Apart Solar de Piedra
Ust'-Talovka - nearest hotel: Металлург
San Luis - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Andalgala,AR,-27.5819,-66.3165,25,Residencial Ayucaba
1,Coronel Du Graty,AR,-27.6804,-60.9146,41,No hotel found
2,Zaragoza,ES,41.6561,-0.8773,49,Hotel Pilar Plaza
3,Bilibino,RU,68.0546,166.4372,47,No hotel found
4,Carnarvon,AU,-24.8667,113.6333,39,No hotel found
5,Konya,TR,38.1667,32.5000,34,No hotel found
6,Karratha,AU,-20.7377,116.8463,44,Karratha International Hotel
7,South Hedland,AU,-20.4089,118.5983,34,Port Haven
8,General Pico,AR,-35.6566,-63.7568,35,Euskadi Hotel
9,Wuwei,CN,37.9281,102.6414,30,Hepingli Boutique Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)